In [11]:
%matplotlib inline

import os

import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

from IPython.display import SVG
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.preprocessing.image import ImageDataGenerator

random_state = 42

In [12]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

x_train = x_train.astype('float32')/255.0
y_train = np.eye(100)[y_train.astype('int32').flatten()]
x_test = x_test.astype('float32')/255.0
y_test = np.eye(100)[y_test.astype('int32').flatten()]

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=10000)

In [13]:
model = Sequential()

model.add(Conv2D(6, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal', input_shape=(32, 32, 3)))  # 32x32x3 -> 28x28x6
model.add(MaxPooling2D(pool_size=(2, 2)))  # 28x28x6 -> 14x14x6
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu',
                 kernel_initializer='he_normal'))  # 14x14x6 -> 10x10x16
model.add(MaxPooling2D(pool_size=(2, 2)))  # 10x10x16 -> 5x5x16

model.add(Flatten())  # 5x5x16 -> 400
model.add(Dense(240, activation='relu',
                kernel_initializer='he_normal'))  # 400 ->240
model.add(Dense(120, activation='relu', kernel_initializer='he_normal'))  # 240 ->120
model.add(Dense(100, activation='softmax'))  # 120 ->100

model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer='adam',
    metrics=['accuracy']
)

In [14]:
datagen = ImageDataGenerator(
    width_shift_range=0.0,  # 3.1.1 左右にずらす
    height_shift_range=0.0,  # 3.1.2 上下にずらす
    horizontal_flip=False,  # 3.1.3 左右反転
    # 3.2.1 Global Contrast Normalization (GCN) 
    samplewise_center=True,
    samplewise_std_normalization=True,
    zca_whitening=True)  # 3.2.2 Zero-phase Component Analysis (ZCA) Whitening (Falseに設定しているのでここでは使用していない)

In [15]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=100),
                    steps_per_epoch=x_train.shape[0] // 100, epochs=30,
                    validation_data=(x_valid, y_valid))

Epoch 1/30
400/400 [==============================] - 11s 27ms/step - loss: 4.0380 - accuracy: 0.0972 - val_loss: 4.4955 - val_accuracy: 0.0364
Epoch 2/30
400/400 [==============================] - 11s 29ms/step - loss: 3.4243 - accuracy: 0.1940 - val_loss: 4.5530 - val_accuracy: 0.0396
Epoch 3/30
400/400 [==============================] - 12s 29ms/step - loss: 3.1435 - accuracy: 0.2401 - val_loss: 4.5367 - val_accuracy: 0.0399
Epoch 4/30
400/400 [==============================] - 12s 30ms/step - loss: 2.9439 - accuracy: 0.2788 - val_loss: 4.9240 - val_accuracy: 0.0196
Epoch 5/30
400/400 [==============================] - 12s 29ms/step - loss: 2.7815 - accuracy: 0.3103 - val_loss: 4.8448 - val_accuracy: 0.0314
Epoch 6/30
400/400 [==============================] - 12s 29ms/step - loss: 2.6368 - accuracy: 0.3371 - val_loss: 4.9634 - val_accuracy: 0.0331
Epoch 7/30
400/400 [==============================] - 12s 30ms/step - loss: 2.5117 - accuracy: 0.3623 - val_loss: 4.8506 - val_accuracy: